In [1]:
!pip install mxnet

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [7]:
from pandas import read_csv
import pandas as pd
from sklearn.model_selection import train_test_split
import autogluon as ag
from autogluon import TabularPrediction as task

In [8]:
test = read_csv("NTI_data/test.csv")
train_target = read_csv("NTI_data/train_target.csv")
transactions_test = read_csv("NTI_data/transactions_test.csv")
transactions_train = read_csv("NTI_data/transactions_train.csv")

In [9]:
#Train
#Признаки, связанные с суммой покупки
agg_features=transactions_train.groupby('client_id')['amount_rur'].agg(['sum','mean','max']).reset_index()

#Количество в каждой категории
amount_of_purchase=transactions_train.groupby(['client_id','small_group'])['amount_rur'].count()
counts_train=amount_of_purchase.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
counts_train=counts_train.fillna(0)
counts_train.columns=['small_group_' + str(i) for i in counts_train.columns]

#Сумма в каждой категории
sum_of_group=transactions_train.groupby(['client_id','small_group'])['amount_rur'].sum()
sum_train=sum_of_group.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
sum_train=sum_train.fillna(0)
sum_train.columns=['small_group_sum_'+str(i) for i in sum_train.columns]

#Минимум в каждой категории
min_of_group=transactions_train.groupby(['client_id','small_group'])['amount_rur'].min()
min_train=sum_of_group.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
min_train=min_train.fillna(0)
min_train.columns=['small_group_min_'+str(i) for i in min_train.columns]

#Максимум в каждой категории
max_of_group=transactions_train.groupby(['client_id','small_group'])['amount_rur'].min()
max_train=sum_of_group.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
max_train=max_train.fillna(0)
max_train.columns=['small_group_max_'+str(i) for i in max_train.columns]

#Средняя сумма в каждой категории
avg_of_group=transactions_train.groupby(['client_id','small_group'])['amount_rur'].mean()
avg_train=avg_of_group.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
avg_train=avg_train.fillna(0)
avg_train.columns=['small_group_avg_'+str(i) for i in avg_train.columns]

#Соединим все в одну таблицу
counts_train=pd.merge(sum_train,counts_train,on='client_id')
counts_train=pd.merge(avg_train,counts_train,on='client_id')
counts_train=pd.merge(min_train,counts_train,on='client_id')
counts_train=pd.merge(max_train,counts_train,on='client_id')
train=pd.merge(train_target,agg_features,on='client_id')
train=pd.merge(train,counts_train.reset_index(),on='client_id')

In [10]:
train_data = train.head(250)
test_data = train.tail(50)

In [1]:
import time
start_time = time.time()

In [11]:
predictor = task.fit(train_data=train_data, label='bins', problem_type="multiclass", auto_stack=True, verbosity=4)
performance = predictor.evaluate(test_data)

No output_directory specified. Models will be saved in: AutogluonModels/ag-20200725_155803\
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20200725_155803\
Train Data Rows:    250
Train Data Columns: 1015
Preprocessing data ...
Train Data Class Count: 4
Feature Generator processed 250 data points with 844 features
Original Features:
	int features: 1
	float features: 843
	object features: 0
Generated Features:
	int features: 0
All Features:
	int features: 1
	float features: 843
	object features: 0
Total time taken in fit_transform: 2.4135665893554688
	Data preprocessing and feature engineering runtime = 2.44s ...
AutoGluon will gauge predictive performance using evaluation metric: accuracy
To change this, specify the eval_metric argument of fit()
AutoGluon will early stop models using evaluation metric: accuracy
Saving AutogluonModels/ag-20200725_155803\learner.pkl
Saving AutogluonModels/ag-20200725_155803\utils\data\X_train.pkl
Saving AutogluonModels/

[1]	train_set's multi_error: 0.53	valid_set's multi_error: 0.54
[2]	train_set's multi_error: 0.405	valid_set's multi_error: 0.5
[3]	train_set's multi_error: 0.375	valid_set's multi_error: 0.48
[4]	train_set's multi_error: 0.285	valid_set's multi_error: 0.48
[5]	train_set's multi_error: 0.235	valid_set's multi_error: 0.5
[6]	train_set's multi_error: 0.215	valid_set's multi_error: 0.48
[7]	train_set's multi_error: 0.2	valid_set's multi_error: 0.48
[8]	train_set's multi_error: 0.165	valid_set's multi_error: 0.48
[9]	train_set's multi_error: 0.145	valid_set's multi_error: 0.48
[10]	train_set's multi_error: 0.13	valid_set's multi_error: 0.5
[11]	train_set's multi_error: 0.12	valid_set's multi_error: 0.5
[12]	train_set's multi_error: 0.095	valid_set's multi_error: 0.5
[13]	train_set's multi_error: 0.07	valid_set's multi_error: 0.5
[14]	train_set's multi_error: 0.05	valid_set's multi_error: 0.52
[15]	train_set's multi_error: 0.05	valid_set's multi_error: 0.48
[16]	train_set's multi_error: 0.0

[161]	train_set's multi_error: 0	valid_set's multi_error: 0.44
[162]	train_set's multi_error: 0	valid_set's multi_error: 0.44
[163]	train_set's multi_error: 0	valid_set's multi_error: 0.46
[164]	train_set's multi_error: 0	valid_set's multi_error: 0.46
[165]	train_set's multi_error: 0	valid_set's multi_error: 0.46
[166]	train_set's multi_error: 0	valid_set's multi_error: 0.46
[167]	train_set's multi_error: 0	valid_set's multi_error: 0.46
[168]	train_set's multi_error: 0	valid_set's multi_error: 0.46
[169]	train_set's multi_error: 0	valid_set's multi_error: 0.46
[170]	train_set's multi_error: 0	valid_set's multi_error: 0.46
[171]	train_set's multi_error: 0	valid_set's multi_error: 0.46
[172]	train_set's multi_error: 0	valid_set's multi_error: 0.46
[173]	train_set's multi_error: 0	valid_set's multi_error: 0.46
[174]	train_set's multi_error: 0	valid_set's multi_error: 0.46
[175]	train_set's multi_error: 0	valid_set's multi_error: 0.46
[176]	train_set's multi_error: 0	valid_set's multi_erro

Training Gradient Boosting Model for 10000 rounds...
with the following hyperparameter settings:
{'num_threads': -1, 'objective': 'multiclass', 'num_classes': 4, 'verbose': -1, 'boosting_type': 'gbdt', 'two_round': True}


[1]	train_set's multi_error: 0.47	valid_set's multi_error: 0.56
[2]	train_set's multi_error: 0.36	valid_set's multi_error: 0.5
[3]	train_set's multi_error: 0.32	valid_set's multi_error: 0.46
[4]	train_set's multi_error: 0.29	valid_set's multi_error: 0.44
[5]	train_set's multi_error: 0.215	valid_set's multi_error: 0.48
[6]	train_set's multi_error: 0.185	valid_set's multi_error: 0.42
[7]	train_set's multi_error: 0.185	valid_set's multi_error: 0.44
[8]	train_set's multi_error: 0.16	valid_set's multi_error: 0.44
[9]	train_set's multi_error: 0.15	valid_set's multi_error: 0.46
[10]	train_set's multi_error: 0.13	valid_set's multi_error: 0.44
[11]	train_set's multi_error: 0.125	valid_set's multi_error: 0.46
[12]	train_set's multi_error: 0.12	valid_set's multi_error: 0.44
[13]	train_set's multi_error: 0.11	valid_set's multi_error: 0.44
[14]	train_set's multi_error: 0.1	valid_set's multi_error: 0.46
[15]	train_set's multi_error: 0.095	valid_set's multi_error: 0.46
[16]	train_set's multi_error: 0

...

Training Gradient Boosting Model for 10000 rounds...
with the following hyperparameter settings:
{'num_threads': -1, 'objective': 'multiclass', 'num_classes': 4, 'verbose': -1, 'boosting_type': 'gbdt', 'two_round': True, 'learning_rate': 0.03, 'num_leaves': 128, 'feature_fraction': 0.9, 'min_data_in_leaf': 3, 'seed_value': 0}


[1]	train_set's multi_error: 0.515	valid_set's multi_error: 0.6
[2]	train_set's multi_error: 0.435	valid_set's multi_error: 0.62
[3]	train_set's multi_error: 0.275	valid_set's multi_error: 0.62
[4]	train_set's multi_error: 0.225	valid_set's multi_error: 0.6
[5]	train_set's multi_error: 0.195	valid_set's multi_error: 0.58
[6]	train_set's multi_error: 0.075	valid_set's multi_error: 0.56
[7]	train_set's multi_error: 0.025	valid_set's multi_error: 0.52
[8]	train_set's multi_error: 0.015	valid_set's multi_error: 0.52
[9]	train_set's multi_error: 0.02	valid_set's multi_error: 0.48
[10]	train_set's multi_error: 0.01	valid_set's multi_error: 0.5
[11]	train_set's multi_error: 0.005	valid_set's multi_error: 0.46
[12]	train_set's multi_error: 0.005	valid_set's multi_error: 0.46
[13]	train_set's multi_error: 0	valid_set's multi_error: 0.44
[14]	train_set's multi_error: 0	valid_set's multi_error: 0.44
[15]	train_set's multi_error: 0	valid_set's multi_error: 0.44
[16]	train_set's multi_error: 0	vali

[134]	train_set's multi_error: 0	valid_set's multi_error: 0.54
[135]	train_set's multi_error: 0	valid_set's multi_error: 0.54
[136]	train_set's multi_error: 0	valid_set's multi_error: 0.54
[137]	train_set's multi_error: 0	valid_set's multi_error: 0.54
[138]	train_set's multi_error: 0	valid_set's multi_error: 0.54
[139]	train_set's multi_error: 0	valid_set's multi_error: 0.54
[140]	train_set's multi_error: 0	valid_set's multi_error: 0.54
[141]	train_set's multi_error: 0	valid_set's multi_error: 0.54
[142]	train_set's multi_error: 0	valid_set's multi_error: 0.54
[143]	train_set's multi_error: 0	valid_set's multi_error: 0.54
[144]	train_set's multi_error: 0	valid_set's multi_error: 0.54
[145]	train_set's multi_error: 0	valid_set's multi_error: 0.54
[146]	train_set's multi_error: 0	valid_set's multi_error: 0.54
[147]	train_set's multi_error: 0	valid_set's multi_error: 0.54
[148]	train_set's multi_error: 0	valid_set's multi_error: 0.54
[149]	train_set's multi_error: 0	valid_set's multi_erro

Saving AutogluonModels/ag-20200725_155803\models\LightGBMClassifierCustom_STACKER_l0\utils\oof.pkl
Saving AutogluonModels/ag-20200725_155803\models\LightGBMClassifierCustom_STACKER_l0\model.pkl
	0.58	 = Validation accuracy score
	51.87s	 = Training runtime
	0.89s	 = Validation runtime
Saving AutogluonModels/ag-20200725_155803\models\trainer.pkl
Loading: AutogluonModels/ag-20200725_155803\models\RandomForestClassifierGini_STACKER_l0\utils\oof.pkl
Loading: AutogluonModels/ag-20200725_155803\models\RandomForestClassifierEntr_STACKER_l0\utils\oof.pkl
Loading: AutogluonModels/ag-20200725_155803\models\ExtraTreesClassifierGini_STACKER_l0\utils\oof.pkl
Loading: AutogluonModels/ag-20200725_155803\models\ExtraTreesClassifierEntr_STACKER_l0\utils\oof.pkl
Loading: AutogluonModels/ag-20200725_155803\models\KNeighborsClassifierUnif_STACKER_l0\utils\oof.pkl
Loading: AutogluonModels/ag-20200725_155803\models\KNeighborsClassifierDist_STACKER_l0\utils\oof.pkl
Loading: AutogluonModels/ag-20200725_155803

Predictive performance on given dataset: accuracy = 0.46


In [2]:
end_time = time.time()

In [3]:
print("Time spent", end_time - start_time)

2.331602096557617
